<a href="https://www.kaggle.com/code/elcientfico/driving-a-taxi-using-tree-methods-and-convolutions?scriptVersionId=148514688" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D,GlobalMaxPooling2D# Input data files are available in the read-only "../input/" directory
from tensorflow.keras.models import Model# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
im_size = Image.open("/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots/1000_1010.png").size
im_size = (im_size[1],im_size[0],3)

In [ ]:
class conv_model:
    def __init__(self):
        pass
def conv_model(ishape = (512,512,3), filters = 256, kernel_s = 3, strides = 3):
    x1  = Input(shape = ishape)
    if len(ishape)<3:
        x2 = tf.expand_dims(x1,-1)
        x2 = Conv2D(filters,kernel_s,strides = strides)(x2)
    else:
        x2 = Conv2D(filters,kernel_s,strides = strides)(x1)
    x3 = Conv2D(filters,kernel_s,strides = strides)(x2)
    x4 = GlobalMaxPooling2D()(x3)
    model = Model(x1,x4)
    return model


In [ ]:
model = conv_model(im_size,64,3,2)
model.summary()

In [ ]:
ss_path = "/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots"
file_list = os.listdir(ss_path)
keys = ["W","S","A","D"]
keys_list = np.array([list(file_name.split("_")[1].split(".")[0]) for file_name in file_list]).astype(int)

LAod images

In [ ]:
image_gen = tf.keras.utils.image_dataset_from_directory(
    ss_path,
    labels=None,
    color_mode='rgb',
    batch_size=16,
    image_size=im_size[:2],
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
from tqdm import tqdm
enc_img_list = model.predict(image_gen,steps = len(image_gen))
# for image_path in tqdm(file_list):
#     im = np.array(Image.open(os.path.join(ss_path,image_path)))
#     im = np.expand_dims(im/255,0)
#     conv_im = model.predict(im,verbose = False)
#     enc_img_list.append(conv_im)


In [ ]:
enc_img_list = np.array(enc_img_list).squeeze()
keys_list = np.array(keys_list)

In [ ]:
import xgboost as xgb

In [ ]:
#["W","S","A","D"]

def mullab_to_mulclass(y):
    if all(y == [1,0,0,0]):
        return [1,0,0,0,0,0,0,0,0]
    elif all(y == [0,1,0,0]):
        return [0,1,0,0,0,0,0,0,0]
    elif all(y == [0,0,1,0]):
        return [0,0,1,0,0,0,0,0,0]
    elif all(y == [0,0,0,1]):
        return [0,0,0,1,0,0,0,0,0]
    elif all(y == [1,1,0,0]):
        return [0,0,0,0,1,0,0,0,0]
    elif all(y == [1,0,1,0]):
        return [0,0,0,0,0,1,0,0,0]
    elif all(y == [0,1,1,0]):
        return [0,0,0,0,0,0,1,0,0]
    elif all(y == [0,0,1,1]):
        return [0,0,0,0,0,0,0,1,0]
    else:
        return [0,0,0,0,0,0,0,0,1]
ylabels = []
for label in keys_list.astype(int):
    ylabels.append(mullab_to_mulclass(label))

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(enc_img_list,ylabels,test_size=.2)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
# rf = MultiOutputClassifier(HistGradientBoostingClassifier(early_stopping = True, verbose  = True,max_iter = 10000, learning_rate = 0.01,
#                                                          n_iter_no_change = 100, scoring = "f1",min_samples_leaf = 1, 
#                                                           max_leaf_nodes = 14))

    
early_stop = xgb.callback.EarlyStopping(
rounds=50, metric_name='logloss', save_best=True,
)

print("traing...")
rf = xgb.XGBClassifier(n_estimators = 1000,callbacks=[early_stop], learning_rate = .01, eval_metric = "logloss",)
rf.fit(x_train,y_train,eval_set=[(x_test, y_test)], verbose = True)
print("Test Acc",rf.score(x_test,y_test),
"Train Acc",rf.score(x_train,y_train))

print("Test F1",f1_score(y_test,rf.predict(x_test),average = "micro"),
"Train F1",f1_score(y_train,rf.predict(x_train),average = "micro"))


traing...
[0]	validation_0-logloss:0.68619
[1]	validation_0-logloss:0.67935
[2]	validation_0-logloss:0.67264
[3]	validation_0-logloss:0.66609
[4]	validation_0-logloss:0.65965
[5]	validation_0-logloss:0.65334
[6]	validation_0-logloss:0.64715
[7]	validation_0-logloss:0.64107
[8]	validation_0-logloss:0.63510
[9]	validation_0-logloss:0.62924
[10]	validation_0-logloss:0.62349
[11]	validation_0-logloss:0.61784
[12]	validation_0-logloss:0.61231
[13]	validation_0-logloss:0.60688
[14]	validation_0-logloss:0.60154
[15]	validation_0-logloss:0.59629
[16]	validation_0-logloss:0.59114
[17]	validation_0-logloss:0.58607
[18]	validation_0-logloss:0.58110
[19]	validation_0-logloss:0.57622
[20]	validation_0-logloss:0.57142
[21]	validation_0-logloss:0.56672
[22]	validation_0-logloss:0.56209
[23]	validation_0-logloss:0.55752
[24]	validation_0-logloss:0.55304
[25]	validation_0-logloss:0.54863
[26]	validation_0-logloss:0.54430
[27]	validation_0-logloss:0.54004
[28]	validation_0-logloss:0.53588
[29]	validatio

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import cv2
import os

# Directory containing the input images
image_dir = "input_images/"

# List of image file names in the directory
image_files = sorted(os.listdir(image_dir))

# Output video file name
output_video = "output_video.mp4"

# Text to overlay and its color
text = "Hello, World!"
text_color = (255, 0, 0)  # Red color (R, G, B)

# Create a drawing context
font = ImageFont.load_default()

# Video settings
frame_rate = 30
frame_size = im_size # Adjust the size as needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec

# Initialize the video writer
out = cv2.VideoWriter(output_video, fourcc, frame_rate, frame_size)

# Iterate through the images and add text, then write to the video
for image_file in image_files:
    # Load the image
    image = Image.open(os.path.join(image_dir, image_file))

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Specify the position for the text (adjust as needed)
    position = (im_size[0]/2, 8*im_size[1]/9)

    # Draw the text on the image
    draw.text(position, text, fill=text_color, font=font,align = "center")

    # Convert the Pillow image to a NumPy array (OpenCV format)
    image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Write the frame to the video
    out.write(image_np)

# Release the video writer
out.release()

print(f"Video saved as {output_video}")
